In [1]:
import numpy as np
import pandas as pd

import data

In [2]:
# PREPROCESSING -- implemented this block from the regression file

# merge all food dataframes into one
full_food = pd.concat([pd.DataFrame(x) for x in data.food.values()], axis=1)
full_food = full_food.loc[:,~full_food.columns.duplicated()]

# For now just don't include the education data at all.
full_df = full_food

# combine food and education dataframes, removing state and county from education
# education = pd.DataFrame(data.education).drop(['State', 'Area name'], axis=1)
# full_df = full_food.merge(education, left_index=True, right_index=True)

# normalize any column that isn't already boolean
bool_cols = [col for col in full_df 
                    if np.isin(full_df[col].dropna().unique(), [0, 1]).all()]
for col in full_df.columns:
    if (col in bool_cols) or (col in ['State', 'County']):
        continue
    full_df[col] = (full_df[col]-full_df[col].min())/(full_df[col].max()-full_df[col].min())

full_df = full_df.drop('County', axis=1)

# one-hot encoding for state
one_hot = pd.get_dummies(full_df['State'])
full_df = full_df.drop('State', axis=1)
full_df = full_df.join(one_hot)

# aggressively just drop any column that has any NaN values in it
full_df = full_df.dropna(axis=1)

print(full_df.shape)

(3140, 165)


In [3]:
#checking the columns for values with low variance
full_df.var()

LACCESS_POP10            0.003356
PCT_LACCESS_POP10        0.040962
LACCESS_LOWI10           0.002248
PCT_LACCESS_LOWI10       0.012801
LACCESS_HHNV10           0.004715
PCT_LACCESS_HHNV10       0.002404
LACCESS_CHILD10          0.002562
PCT_LACCESS_CHILD10      0.020410
LACCESS_SENIORS10        0.006844
PCT_LACCESS_SENIORS10    0.020846
GROC09                   0.001546
GROC14                   0.001385
GROCPTH09                0.005333
GROCPTH14                0.004849
SUPERC09                 0.001641
SUPERC14                 0.001771
SUPERCPTH09              0.006680
SUPERCPTH14              0.008755
CONVS09                  0.002030
CONVS14                  0.002006
CONVSPTH09               0.009847
CONVSPTH14               0.004571
SPECS09                  0.001165
SPECS14                  0.001426
SPECSPTH09               0.003076
SPECSPTH14               0.002776
SNAPS12                  0.001180
SNAPSPTH12               0.003356
WICS08                   0.001507
WICS12        

In [4]:
from sklearn.feature_selection import VarianceThreshold
#Removing  features with low variance (threshold =0.01)
constant_filter = VarianceThreshold(threshold=0.01)
print("Original column length" +" "+ str(len(full_df.columns)))
constant_filter.fit(full_df)
print("Length of columns after setting variance threshold"+ " " + str(len(full_df.columns[constant_filter.get_support()])))

Original column length 165
Length of columns after setting variance threshold 89


In [5]:
full_df = pd.DataFrame(constant_filter.transform(full_df))

In [6]:
print(full_df.shape)

(3140, 89)


In [7]:
full_df.corr()

0         1         2         3         4         5         6   \
0   1.000000  0.903093  0.961471  0.920769 -0.020661 -0.048898 -0.254132   
1   0.903093  1.000000  0.889470  0.831444  0.013468  0.018909 -0.120557   
2   0.961471  0.889470  1.000000  0.827881 -0.007775 -0.033060 -0.244107   
3   0.920769  0.831444  0.827881  1.000000 -0.061985 -0.077934 -0.258062   
4  -0.020661  0.013468 -0.007775 -0.061985  1.000000  0.805248  0.141644   
5  -0.048898  0.018909 -0.033060 -0.077934  0.805248  1.000000  0.271678   
6  -0.254132 -0.120557 -0.244107 -0.258062  0.141644  0.271678  1.000000   
7  -0.203849 -0.086977 -0.203205 -0.213076  0.138634  0.248409  0.916489   
8   0.182952  0.108287  0.159387  0.172358 -0.046685 -0.128209 -0.467188   
9  -0.184014 -0.146430 -0.195378 -0.135112 -0.425072 -0.304032  0.429891   
10 -0.230342 -0.208179 -0.224812 -0.212154 -0.277671 -0.323772  0.461077   
11 -0.041852 -0.077763 -0.044080 -0.020617  0.084827  0.017024 -0.075694   
12  0.023382  0.018779 -0.000520  0.072622 -0.104863  0.028943  0.035796   
13 -0.117501 -0.095762 -0.121129 -0.121710  0.166194  0.164453  0.266088   
14 -0.059415 -0.046348 -0.056852 -0.068209  0.129581  0.135712  0.216959   
15  0.016335 -0.008968 -0.000192  0.018955  0.006934 -0.095186  0.034490   
16 -0.019385 -0.028899 -0.043971  0.006278  0.000051 -0.069033  0.153555   
17  0.013639  0.037943  0.020495  0.029256 -0.151412 -0.062386  0.270215   
18  0.064808  0.132420  0.101467  0.071272  0.057555  0.397400  0.172996   
19  0.095963  0.153126  0.131056  0.103482  0.064366  0.393778  0.141258   
20  0.147920  0.073520  0.129413  0.151743  0.020305 -0.129525 -0.216993   
21 -0.061958  0.065614 -0.037619 -0.085579  0.427318  0.651752  0.631074   
22 -0.107041  0.012005 -0.089903 -0.124092  0.406636  0.642353  0.634091   
23 -0.124929 -0.135938 -0.141780 -0.109679 -0.016033  0.034157  0.065144   
24 -0.006720 -0.008143 -0.016041 -0.008166 -0.297714 -0.283577 -0.035396   
25 -0.031049  0.063229 -0.000306 -0.082472  0.502980  0.565989  0.477379   
26 -0.041711  0.055759 -0.015867 -0.083188  0.428174  0.549928  0.495610   
27  0.009090 -0.046130 -0.014538  0.050270 -0.381513 -0.364837 -0.277643   
28  0.286153  0.270767  0.280833  0.309523 -0.228670 -0.025199 -0.233868   
29  0.255124  0.254369  0.257354  0.262410 -0.145375  0.114626 -0.148049   
..       ...       ...       ...       ...       ...       ...       ...   
59 -0.070297 -0.032148 -0.056514 -0.095772  0.258154  0.194971  0.271874   
60 -0.041016 -0.055189 -0.039090 -0.012935 -0.284897 -0.275420 -0.089791   
61  0.026069  0.043986  0.044882  0.010135 -0.087545 -0.195382 -0.017998   
62 -0.035274 -0.045330 -0.039647 -0.034626 -0.022423  0.021073 -0.038252   
63 -0.079303 -0.085502 -0.071965 -0.078023  0.020361  0.025635 -0.063329   
64  0.113082  0.094062  0.120784  0.119177 -0.059186  0.021568 -0.237752   
65 -0.127503 -0.102914 -0.119876 -0.116311  0.050727  0.197616  0.232689   
66  0.000149  0.022226  0.007269 -0.033398 -0.016623  0.093518  0.167765   
67 -0.059882 -0.059342 -0.072350 -0.038393 -0.255221 -0.158613  0.146352   
68  0.027223 -0.009580  0.029179  0.034235 -0.109717 -0.168177 -0.237106   
69 -0.035798 -0.015360 -0.032262 -0.027662 -0.036559 -0.080191  0.028043   
70 -0.030993  0.036331 -0.017244 -0.049385 -0.089845  0.057465  0.306546   
71  0.143084  0.151591  0.125463  0.150039 -0.128916 -0.147696 -0.096660   
72 -0.085505 -0.067384 -0.085531 -0.079797  0.195892  0.104064  0.091037   
73  0.165430  0.108907  0.131337  0.199836 -0.202977 -0.023861 -0.248191   
74  0.180995  0.152385  0.176759  0.214925 -0.065437 -0.068492 -0.274129   
75  0.091699  0.121790  0.084588  0.091714  0.101901  0.086250  0.169076   
76 -0.051208 -0.070699 -0.059585 -0.052018 -0.226284 -0.211386  0.023503   
77 -0.060806 -0.069831 -0.058766 -0.058807 -0.033969  0.106575  0.021694   
78  0.033673  0.049928  0.038609  0.025989  0.057897  0.047019  0.032910   
79  0.015339  0.020743  0.004908  0.030186  0.076944 -0.09

In [8]:
#Removing features with high correlation
correlated_features = set()
correlation_matrix = full_df.corr()

In [9]:
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [10]:
print(correlated_features)

{32, 1, 2, 3, 35, 5, 37, 7, 38, 59, 43, 14, 19, 22, 56, 27, 29}


In [11]:
print(len(correlated_features))

17


In [12]:
full_df.drop(labels=correlated_features, axis=1, inplace=True)
print(full_df.shape)

(3140, 72)


In [13]:
#Using Random forest for feature selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import minmax_scale

model = RandomForestRegressor(random_state=1, max_depth=10,n_estimators=100)
full_df=pd.get_dummies(full_df)


X = full_df.values
y = minmax_scale(data.education["Percent of adults with a bachelor's degree or higher, 2014-18"].values)

model.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [22]:
import matplotlib.pyplot as plt
features = full_df.columns
importances = model.feature_importances_
indices = np.argsort(importances)[-15:]  # top 10 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [25]:
from sklearn.feature_selection import SelectFromModel
#feature = SelectFromModel(model)
sfm = SelectFromModel(model, threshold=1e-4, prefit=True)
Fit = sfm.transform(full_df)
print(full_df.shape)

(3140, 72)


In [27]:
'''
DIMENSIONALITY REDUCTION - BACKWARD FEATURE ELIMINATION
'''

# importing required libraries
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE



# create the object of the model
lreg = LinearRegression()


# specify the number of  features to select 
rfe = RFE(lreg, 50)

# fit the model
rfe = rfe.fit(X, y)

print('\n\nFEATUERS SELECTED\n\n')
print(rfe.support_)

print('\n\nRANKING OF FEATURES\n\n')
print(rfe.ranking_)



FEATUERS SELECTED


[False False  True  True  True  True  True False  True  True  True  True
  True  True  True False  True False  True  True  True  True  True  True
  True  True  True  True  True False False False False False False False
 False False False False  True  True  True False  True  True  True False
  True  True  True  True False False  True  True  True  True  True  True
 False  True False  True  True  True  True  True  True  True  True  True]


RANKING OF FEATURES


[22  5  1  1  1  1  1  6  1  1  1  1  1  1  1 11  1  2  1  1  1  1  1  1
  1  1  1  1  1 15 13 17 20 16 14 21 19 18 23  3  1  1  1  7  1  1  1  4
  1  1  1  1  8  9  1  1  1  1  1  1 10  1 12  1  1  1  1  1  1  1  1  1]


In [ ]:
#BoxPlot to assess the skewness of data


#fig1, ax1 = plt.subplots()
#ax1.set_title('Box plot of all columns')
#ax1.boxplot(full_df)

#plt.show()